In [1]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt 
import matplotlib.style as stl 
from tqdm import tqdm
from tensorflow import keras 
from keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
import string
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, LSTM, Embedding,Dropout,SpatialDropout1D,Conv1D,MaxPooling1D,GRU,BatchNormalization
import warnings
warnings.filterwarnings('ignore')
stl.use('ggplot')
import re

In [2]:
data=pd.read_csv("../input/superheroes-nlp-dataset/superheroes_nlp_dataset.csv")

In [3]:
data.head()

,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,durability_score,...,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
0,3-D Man,"Delroy Garrett, Jr.","Delroy Garrett, Jr.",6,"Delroy Garrett, Jr. grew up to become a track ...",NaN,85,30,60,60,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,514A (Gotham),Bruce Wayne,NaN,10,He was one of the many prisoners of Indian Hil...,NaN,100,20,30,50,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
2,A-Bomb,Richard Milhouse Jones,Richard Milhouse Jones,20,"Richard ""Rick"" Jones was orphaned at a young ...","On rare occasions, and through unusual circu...",80,100,80,100,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
3,Aa,Aa,NaN,12,Aa is one of the more passive members of the P...,NaN,80,50,55,45,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Aaron Cash,Aaron Cash,Aaron Cash,5,Aaron Cash is the head of security at Arkham A...,NaN,80,10,25,40,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
data.shape

(1450, 81)

In [5]:
data.isnull().sum()

name                    2
real_name             149
full_name             494
overall_score           0
history_text           90
                     ... 
has_super_speed        67
has_durability         67
has_stamina            67
has_agility            67
has_super_strength     67
Length: 81, dtype: int64

In [6]:
data=data.fillna('')

In [7]:
data["hero_details"] = str(data["name"]) + " " + str(data["real_name"]) + " " + str(data["full_name"])+ " " + str(data["history_text"])

In [8]:
data.head()

,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,durability_score,...,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength,hero_details
0,3-D Man,"Delroy Garrett, Jr.","Delroy Garrett, Jr.",6,"Delroy Garrett, Jr. grew up to become a track ...",,85,30,60,60,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0 3-D Man\n1 514A (Gotham)\n...
1,514A (Gotham),Bruce Wayne,,10,He was one of the many prisoners of Indian Hil...,,100,20,30,50,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0 3-D Man\n1 514A (Gotham)\n...
2,A-Bomb,Richard Milhouse Jones,Richard Milhouse Jones,20,"Richard ""Rick"" Jones was orphaned at a young ...","On rare occasions, and through unusual circu...",80,100,80,100,...,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0 3-D Man\n1 514A (Gotham)\n...
3,Aa,Aa,,12,Aa is one of the more passive members of the P...,,80,50,55,45,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0 3-D Man\n1 514A (Gotham)\n...
4,Aaron Cash,Aaron Cash,Aaron Cash,5,Aaron Cash is the head of security at Arkham A...,,80,10,25,40,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0 3-D Man\n1 514A (Gotham)\n...


In [9]:
def clean(data):
    data1=re.sub(r'[?|$|(),"".@#=><|!]Â&*/',r' ',data)
    data2=re.sub(r'[^a-z A-Z]',r' ',data1)
    data3=data2.lower()
    data4=re.sub(r'\b\w{1,3}\b', ' ',data3)
    data5=re.sub(r'https?://\S+|www\.\S+',r' ',data4)
    data6=re.sub(r' +', ' ',data5)
    return(data6)

In [10]:
# from tqdm import tqdm

# for i in tqdm(range(data.shape[0])):
#     ndata[i] = clean(data[i])

In [11]:
data

,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,durability_score,...,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength,hero_details
0,3-D Man,"Delroy Garrett, Jr.","Delroy Garrett, Jr.",6,"Delroy Garrett, Jr. grew up to become a track ...",,85,30,60,60,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0 3-D Man\n1 514A (Gotham)\n...
1,514A (Gotham),Bruce Wayne,,10,He was one of the many prisoners of Indian Hil...,,100,20,30,50,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0 3-D Man\n1 514A (Gotham)\n...
2,A-Bomb,Richard Milhouse Jones,Richard Milhouse Jones,20,"Richard ""Rick"" Jones was orphaned at a young ...","On rare occasions, and through unusual circu...",80,100,80,100,...,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0 3-D Man\n1 514A (Gotham)\n...
3,Aa,Aa,,12,Aa is one of the more passive members of the P...,,80,50,55,45,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0 3-D Man\n1 514A (Gotham)\n...
4,Aaron Cash,Aaron Cash,Aaron Cash,5,Aaron Cash is the head of security at Arkham A...,,80,10,25,40,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0 3-D Man\n1 514A (Gotham)\n...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1445,Zatanna,Zatanna Zatara,Zatanna Zatara,10,Zatanna is the daughter of adventurer John Zat...,Zatanna is genetically talented with her magi...,90,10,25,30,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0 3-D Man\n1 514A (Gotham)\n...
1446,Zero,DWN-∞: Zero,DWN-∞: Zero,18,Zero was created by the late Dr. Albert Wily ...,,80,100,100,100,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0 3-D Man\n1 514A (Gotham)\n...
1447,Zoom (New 52),Hunter Zolomon,,20,"Hunter Zolomon is better known as Zoom, a spee...",After tricking Barry Allen and Wally West into...,95,50,100,75,...,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0 3-D Man\n1 514A (Gotham)\n...
1448,Zoom,Hunter Zolomon,Hunter Zolomon,9,Hunter Zolomon had a troubled relationship wi...,"Zoom is able to alter time, to make himself ev...",75,10,100,30,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0 3-D Man\n1 514A (Gotham)\n...
